# Credit Card Default Prediction: Data Preparation & Feature Engineering

## Assignment Overview
This notebook focuses on preparing and engineering features for predicting credit card defaults using a **cost-aware, fair, and explainable** machine learning approach.

**Dataset**: [Default of Credit Card Clients Dataset](https://www.kaggle.com/uciml/default-of-credit-card-clients-dataset) from Kaggle

## Learning Objectives
- Understand credit card default data characteristics
- Perform thorough data quality assessment
- Create meaningful features for ML modeling
- Prepare data for fair and explainable ML approaches
- Analyze potential fairness concerns in financial data

---

## 1. Setup and Imports

In [ ]:
# Data manipulation and analysis
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical analysis
from scipy import stats
from scipy.stats import chi2_contingency

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("Libraries imported successfully!")

## 2. Data Loading

### Dataset Information
The dataset contains information on default payments, demographic factors, credit data, payment history, and bill statements of credit card clients in Taiwan from April 2005 to September 2005.

**To download the dataset:**
1. Visit: https://www.kaggle.com/uciml/default-of-credit-card-clients-dataset
2. Download `UCI_Credit_Card.csv`
3. Place it in the same directory as this notebook or update the path below

**Alternative**: You can also download it programmatically using the Kaggle API (requires setup)

In [ ]:
# Load the dataset
# Update this path if your data is located elsewhere
data_path = 'UCI_Credit_Card.csv'

try:
    df = pd.read_csv(data_path)
    print(f"✓ Dataset loaded successfully!")
    print(f"  Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
except FileNotFoundError:
    print("❌ File not found. Please download the dataset from Kaggle and place it in the correct location.")
    print("   Expected path:", data_path)

## 3. Initial Data Exploration

In [ ]:
# Display first few rows
print("First 5 rows of the dataset:")
df.head()

In [ ]:
# Dataset information
print("Dataset Information:")
print("=" * 80)
df.info()

In [ ]:
# Statistical summary
print("Statistical Summary:")
print("=" * 80)
df.describe()

### Understanding the Features

**Target Variable:**
- `default.payment.next.month`: Default payment (1=yes, 0=no)

**Demographic Features:**
- `ID`: Customer ID
- `SEX`: Gender (1=male, 2=female)
- `EDUCATION`: Education level (1=graduate school, 2=university, 3=high school, 4=others)
- `MARRIAGE`: Marital status (1=married, 2=single, 3=others)
- `AGE`: Age in years

**Credit Information:**
- `LIMIT_BAL`: Amount of given credit (NT dollar)

**Payment History (Sep 2005 - Apr 2005):**
- `PAY_0` to `PAY_6`: Repayment status for each month
  - -1=pay duly, 1=payment delay for one month, 2=payment delay for two months, etc.

**Bill Statements (Sep 2005 - Apr 2005):**
- `BILL_AMT1` to `BILL_AMT6`: Bill statement amount for each month

**Payment Amounts (Sep 2005 - Apr 2005):**
- `PAY_AMT1` to `PAY_AMT6`: Previous payment amount for each month

In [ ]:
# Check column names
print("Column Names:")
print("=" * 80)
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

## 4. Data Quality Assessment

In [ ]:
# Check for missing values
print("Missing Values Analysis:")
print("=" * 80)
missing_data = df.isnull().sum()
missing_percent = 100 * df.isnull().sum() / len(df)
missing_table = pd.DataFrame({
    'Missing Count': missing_data,
    'Percentage': missing_percent
})
missing_table = missing_table[missing_table['Missing Count'] > 0].sort_values(
    'Missing Count', ascending=False
)

if len(missing_table) == 0:
    print("✓ No missing values found!")
else:
    print(missing_table)

In [ ]:
# Check for duplicate records
duplicates = df.duplicated().sum()
print(f"Duplicate Records: {duplicates}")

# Check for duplicate IDs
duplicate_ids = df['ID'].duplicated().sum()
print(f"Duplicate IDs: {duplicate_ids}")

In [ ]:
# Check target variable distribution
print("Target Variable Distribution:")
print("=" * 80)
target_col = 'default.payment.next.month'
target_counts = df[target_col].value_counts().sort_index()
target_pct = df[target_col].value_counts(normalize=True).sort_index() * 100

target_summary = pd.DataFrame({
    'Count': target_counts,
    'Percentage': target_pct
})
target_summary.index = ['No Default (0)', 'Default (1)']
print(target_summary)

# Visualize
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
target_counts.plot(kind='bar', ax=ax[0], color=['#2ecc71', '#e74c3c'])
ax[0].set_title('Target Variable Distribution', fontsize=12, fontweight='bold')
ax[0].set_xlabel('Default Status')
ax[0].set_ylabel('Count')
ax[0].set_xticklabels(['No Default', 'Default'], rotation=0)

ax[1].pie(target_counts, labels=['No Default', 'Default'], autopct='%1.1f%%',
          colors=['#2ecc71', '#e74c3c'], startangle=90)
ax[1].set_title('Target Variable Proportion', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# Calculate imbalance ratio
imbalance_ratio = target_counts.max() / target_counts.min()
print(f"\nClass Imbalance Ratio: {imbalance_ratio:.2f}:1")
if imbalance_ratio > 2:
    print("⚠️  Dataset is imbalanced. Consider using techniques like SMOTE, class weights, or stratified sampling.")

## 5. Data Cleaning and Preprocessing

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

# Rename target variable for easier access
df_clean = df_clean.rename(columns={'default.payment.next.month': 'default'})

print("✓ Dataset copied and target variable renamed")

In [ ]:
# Examine categorical variables
print("Categorical Variable Analysis:")
print("=" * 80)

categorical_cols = ['SEX', 'EDUCATION', 'MARRIAGE']
for col in categorical_cols:
    print(f"\n{col}:")
    print(df_clean[col].value_counts().sort_index())

In [ ]:
# Clean EDUCATION variable (0, 5, 6 should be combined with 4 'others')
print("Cleaning EDUCATION variable...")
print(f"Before: {df_clean['EDUCATION'].value_counts().sort_index().to_dict()}")

df_clean['EDUCATION'] = df_clean['EDUCATION'].replace({0: 4, 5: 4, 6: 4})

print(f"After: {df_clean['EDUCATION'].value_counts().sort_index().to_dict()}")
print("✓ EDUCATION cleaned")

In [ ]:
# Clean MARRIAGE variable (0 should be combined with 3 'others')
print("Cleaning MARRIAGE variable...")
print(f"Before: {df_clean['MARRIAGE'].value_counts().sort_index().to_dict()}")

df_clean['MARRIAGE'] = df_clean['MARRIAGE'].replace({0: 3})

print(f"After: {df_clean['MARRIAGE'].value_counts().sort_index().to_dict()}")
print("✓ MARRIAGE cleaned")

In [ ]:
# Examine payment status variables
print("Payment Status Variables Analysis:")
print("=" * 80)

pay_cols = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
for col in pay_cols:
    print(f"\n{col} unique values: {sorted(df_clean[col].unique())}")

In [ ]:
# Check for outliers in numerical columns
print("Outlier Detection (using IQR method):")
print("=" * 80)

numerical_cols = ['LIMIT_BAL', 'AGE'] + [f'BILL_AMT{i}' for i in range(1, 7)] + [f'PAY_AMT{i}' for i in range(1, 7)]

outlier_summary = []
for col in numerical_cols:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df_clean[(df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)][col]
    outlier_pct = (len(outliers) / len(df_clean)) * 100
    
    outlier_summary.append({
        'Column': col,
        'Outlier Count': len(outliers),
        'Percentage': outlier_pct
    })

outlier_df = pd.DataFrame(outlier_summary)
outlier_df = outlier_df[outlier_df['Outlier Count'] > 0].sort_values('Outlier Count', ascending=False)
print(outlier_df.to_string(index=False))

print("\n⚠️  Note: Outliers in financial data may represent genuine high-value transactions.")
print("   We'll keep them for now but may need to handle them during modeling.")

## 6. Exploratory Data Analysis (EDA)

### 6.1 Demographic Analysis

In [ ]:
# Age distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Overall age distribution
axes[0].hist(df_clean['AGE'], bins=30, edgecolor='black', alpha=0.7)
axes[0].set_title('Age Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Frequency')
axes[0].axvline(df_clean['AGE'].mean(), color='red', linestyle='--', label=f'Mean: {df_clean["AGE"].mean():.1f}')
axes[0].axvline(df_clean['AGE'].median(), color='green', linestyle='--', label=f'Median: {df_clean["AGE"].median():.1f}')
axes[0].legend()

# Age distribution by default status
df_clean[df_clean['default'] == 0]['AGE'].hist(bins=30, alpha=0.6, label='No Default', ax=axes[1])
df_clean[df_clean['default'] == 1]['AGE'].hist(bins=30, alpha=0.6, label='Default', ax=axes[1])
axes[1].set_title('Age Distribution by Default Status', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Age')
axes[1].set_ylabel('Frequency')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Gender, Education, and Marriage analysis
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# SEX
sex_default = pd.crosstab(df_clean['SEX'], df_clean['default'], normalize='index') * 100
sex_default.plot(kind='bar', ax=axes[0, 0], color=['#2ecc71', '#e74c3c'])
axes[0, 0].set_title('Default Rate by Gender', fontsize=11, fontweight='bold')
axes[0, 0].set_xlabel('Gender (1=Male, 2=Female)')
axes[0, 0].set_ylabel('Percentage')
axes[0, 0].set_xticklabels(['Male', 'Female'], rotation=0)
axes[0, 0].legend(['No Default', 'Default'])

sex_counts = df_clean['SEX'].value_counts().sort_index()
axes[1, 0].bar(['Male', 'Female'], sex_counts.values, color=['#3498db', '#e91e63'])
axes[1, 0].set_title('Gender Distribution', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Count')

# EDUCATION
edu_default = pd.crosstab(df_clean['EDUCATION'], df_clean['default'], normalize='index') * 100
edu_default.plot(kind='bar', ax=axes[0, 1], color=['#2ecc71', '#e74c3c'])
axes[0, 1].set_title('Default Rate by Education', fontsize=11, fontweight='bold')
axes[0, 1].set_xlabel('Education Level')
axes[0, 1].set_ylabel('Percentage')
axes[0, 1].set_xticklabels(['Grad School', 'University', 'High School', 'Others'], rotation=45)
axes[0, 1].legend(['No Default', 'Default'])

edu_counts = df_clean['EDUCATION'].value_counts().sort_index()
axes[1, 1].bar(['Grad School', 'University', 'High School', 'Others'], edu_counts.values)
axes[1, 1].set_title('Education Distribution', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Count')
axes[1, 1].tick_params(axis='x', rotation=45)

# MARRIAGE
mar_default = pd.crosstab(df_clean['MARRIAGE'], df_clean['default'], normalize='index') * 100
mar_default.plot(kind='bar', ax=axes[0, 2], color=['#2ecc71', '#e74c3c'])
axes[0, 2].set_title('Default Rate by Marital Status', fontsize=11, fontweight='bold')
axes[0, 2].set_xlabel('Marital Status')
axes[0, 2].set_ylabel('Percentage')
axes[0, 2].set_xticklabels(['Married', 'Single', 'Others'], rotation=0)
axes[0, 2].legend(['No Default', 'Default'])

mar_counts = df_clean['MARRIAGE'].value_counts().sort_index()
axes[1, 2].bar(['Married', 'Single', 'Others'], mar_counts.values, color=['#9b59b6', '#f39c12', '#34495e'])
axes[1, 2].set_title('Marital Status Distribution', fontsize=11, fontweight='bold')
axes[1, 2].set_ylabel('Count')

plt.tight_layout()
plt.show()

### 6.2 Credit Limit Analysis

In [ ]:
# Credit limit analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution of credit limit
axes[0].hist(df_clean['LIMIT_BAL'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Credit Limit Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Credit Limit (NT$)')
axes[0].set_ylabel('Frequency')
axes[0].axvline(df_clean['LIMIT_BAL'].mean(), color='red', linestyle='--', 
                label=f'Mean: ${df_clean["LIMIT_BAL"].mean():,.0f}')
axes[0].legend()

# Credit limit by default status
df_clean.boxplot(column='LIMIT_BAL', by='default', ax=axes[1])
axes[1].set_title('Credit Limit by Default Status', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Default Status (0=No, 1=Yes)')
axes[1].set_ylabel('Credit Limit (NT$)')
plt.suptitle('')  # Remove the automatic title

plt.tight_layout()
plt.show()

# Statistics
print("Credit Limit Statistics by Default Status:")
print("=" * 80)
print(df_clean.groupby('default')['LIMIT_BAL'].describe())

### 6.3 Payment History Analysis

In [ ]:
# Payment status analysis
pay_cols = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, col in enumerate(pay_cols):
    pay_default = pd.crosstab(df_clean[col], df_clean['default'], normalize='index') * 100
    pay_default[1].plot(kind='bar', ax=axes[i], color='#e74c3c')
    axes[i].set_title(f'Default Rate by {col}', fontsize=10, fontweight='bold')
    axes[i].set_xlabel('Payment Status')
    axes[i].set_ylabel('Default Rate (%)')
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Average payment status by default
avg_pay_status = df_clean.groupby('default')[pay_cols].mean()

fig, ax = plt.subplots(figsize=(12, 5))
avg_pay_status.T.plot(kind='bar', ax=ax, color=['#2ecc71', '#e74c3c'])
ax.set_title('Average Payment Status by Default Status', fontsize=12, fontweight='bold')
ax.set_xlabel('Month')
ax.set_ylabel('Average Payment Status')
ax.legend(['No Default', 'Default'])
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

### 6.4 Bill Amount and Payment Amount Analysis

In [ ]:
# Average bill amounts over time
bill_cols = ['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']
pay_amt_cols = ['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bill amounts
avg_bill = df_clean.groupby('default')[bill_cols].mean()
avg_bill.T.plot(ax=axes[0], marker='o', linewidth=2)
axes[0].set_title('Average Bill Amount Over Time', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Month (1=Sep, 6=Apr)')
axes[0].set_ylabel('Average Bill Amount (NT$)')
axes[0].legend(['No Default', 'Default'])
axes[0].grid(True, alpha=0.3)

# Payment amounts
avg_pay = df_clean.groupby('default')[pay_amt_cols].mean()
avg_pay.T.plot(ax=axes[1], marker='o', linewidth=2)
axes[1].set_title('Average Payment Amount Over Time', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Month (1=Sep, 6=Apr)')
axes[1].set_ylabel('Average Payment Amount (NT$)')
axes[1].legend(['No Default', 'Default'])
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 6.5 Correlation Analysis

In [ ]:
# Correlation matrix for key features
key_features = ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 
                'BILL_AMT1', 'BILL_AMT2', 'PAY_AMT1', 'PAY_AMT2', 'default']

plt.figure(figsize=(12, 10))
correlation_matrix = df_clean[key_features].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Correlation Matrix - Key Features', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Features most correlated with default
print("\nFeatures Most Correlated with Default:")
print("=" * 80)
default_corr = correlation_matrix['default'].sort_values(ascending=False)
print(default_corr)

## 7. Feature Engineering

### 7.1 Payment Behavior Features

In [ ]:
# Create a copy for feature engineering
df_featured = df_clean.copy()

print("Creating payment behavior features...")

# 1. Average payment status (higher = more delays)
df_featured['avg_payment_status'] = df_featured[pay_cols].mean(axis=1)

# 2. Maximum payment delay
df_featured['max_payment_delay'] = df_featured[pay_cols].max(axis=1)

# 3. Count of months with payment delay (payment status > 0)
df_featured['num_delayed_payments'] = (df_featured[pay_cols] > 0).sum(axis=1)

# 4. Count of months with on-time payment (payment status <= 0)
df_featured['num_ontime_payments'] = (df_featured[pay_cols] <= 0).sum(axis=1)

# 5. Payment status trend (is it getting worse?)
# Positive slope means getting worse over time
df_featured['payment_status_trend'] = df_featured['PAY_0'] - df_featured['PAY_6']

# 6. Recent payment behavior (last 2 months average)
df_featured['recent_payment_status'] = df_featured[['PAY_0', 'PAY_2']].mean(axis=1)

print("✓ Payment behavior features created")

### 7.2 Credit Utilization Features

In [ ]:
print("Creating credit utilization features...")

# 7. Average bill amount
df_featured['avg_bill_amt'] = df_featured[bill_cols].mean(axis=1)

# 8. Maximum bill amount
df_featured['max_bill_amt'] = df_featured[bill_cols].max(axis=1)

# 9. Bill amount volatility (standard deviation)
df_featured['bill_amt_volatility'] = df_featured[bill_cols].std(axis=1)

# 10. Credit utilization ratio (avg bill / credit limit)
df_featured['credit_utilization'] = df_featured['avg_bill_amt'] / df_featured['LIMIT_BAL']
df_featured['credit_utilization'] = df_featured['credit_utilization'].clip(upper=2)  # Cap at 200%

# 11. Recent credit utilization (last month)
df_featured['recent_credit_utilization'] = df_featured['BILL_AMT1'] / df_featured['LIMIT_BAL']
df_featured['recent_credit_utilization'] = df_featured['recent_credit_utilization'].clip(upper=2)

# 12. Bill amount trend (increasing or decreasing?)
df_featured['bill_amt_trend'] = df_featured['BILL_AMT1'] - df_featured['BILL_AMT6']

print("✓ Credit utilization features created")

### 7.3 Payment Capacity Features

In [ ]:
print("Creating payment capacity features...")

# 13. Average payment amount
df_featured['avg_payment_amt'] = df_featured[pay_amt_cols].mean(axis=1)

# 14. Payment to bill ratio (how much of the bill is being paid?)
# Higher is better (paying more of the bill)
df_featured['payment_to_bill_ratio'] = df_featured['avg_payment_amt'] / (df_featured['avg_bill_amt'] + 1)
df_featured['payment_to_bill_ratio'] = df_featured['payment_to_bill_ratio'].clip(upper=5)

# 15. Recent payment to bill ratio
df_featured['recent_payment_to_bill'] = df_featured['PAY_AMT1'] / (df_featured['BILL_AMT1'] + 1)
df_featured['recent_payment_to_bill'] = df_featured['recent_payment_to_bill'].clip(upper=5)

# 16. Payment amount volatility
df_featured['payment_amt_volatility'] = df_featured[pay_amt_cols].std(axis=1)

# 17. Count of zero payments (months with no payment)
df_featured['num_zero_payments'] = (df_featured[pay_amt_cols] == 0).sum(axis=1)

# 18. Payment consistency (inverse of coefficient of variation)
payment_mean = df_featured[pay_amt_cols].mean(axis=1)
payment_std = df_featured[pay_amt_cols].std(axis=1)
df_featured['payment_consistency'] = 1 / (1 + payment_std / (payment_mean + 1))

print("✓ Payment capacity features created")

### 7.4 Categorical Encoding

In [ ]:
print("Creating categorical features...")

# Create meaningful labels for categorical variables
# SEX
df_featured['SEX_male'] = (df_featured['SEX'] == 1).astype(int)
df_featured['SEX_female'] = (df_featured['SEX'] == 2).astype(int)

# EDUCATION - One-hot encoding
df_featured['EDU_graduate'] = (df_featured['EDUCATION'] == 1).astype(int)
df_featured['EDU_university'] = (df_featured['EDUCATION'] == 2).astype(int)
df_featured['EDU_high_school'] = (df_featured['EDUCATION'] == 3).astype(int)
df_featured['EDU_others'] = (df_featured['EDUCATION'] == 4).astype(int)

# MARRIAGE - One-hot encoding
df_featured['MAR_married'] = (df_featured['MARRIAGE'] == 1).astype(int)
df_featured['MAR_single'] = (df_featured['MARRIAGE'] == 2).astype(int)
df_featured['MAR_others'] = (df_featured['MARRIAGE'] == 3).astype(int)

print("✓ Categorical features created")

### 7.5 Age-based Features

In [ ]:
print("Creating age-based features...")

# Age groups
df_featured['age_group'] = pd.cut(df_featured['AGE'], 
                                   bins=[0, 25, 35, 45, 55, 100],
                                   labels=['<25', '25-35', '35-45', '45-55', '55+'])

# One-hot encode age groups
age_dummies = pd.get_dummies(df_featured['age_group'], prefix='age')
df_featured = pd.concat([df_featured, age_dummies], axis=1)

# Credit limit per year of age (as a rough measure of financial capacity relative to age)
df_featured['credit_per_age'] = df_featured['LIMIT_BAL'] / df_featured['AGE']

print("✓ Age-based features created")

### 7.6 Interaction Features

In [ ]:
print("Creating interaction features...")

# High utilization + payment delays (risky combination)
df_featured['high_util_delayed'] = (
    (df_featured['credit_utilization'] > 0.7) & 
    (df_featured['num_delayed_payments'] > 2)
).astype(int)

# Low payment ratio + high bills (another risky combination)
df_featured['low_payment_high_bill'] = (
    (df_featured['payment_to_bill_ratio'] < 0.1) & 
    (df_featured['avg_bill_amt'] > df_featured['avg_bill_amt'].median())
).astype(int)

print("✓ Interaction features created")

### 7.7 Feature Summary

In [ ]:
# List all new features created
original_features = set(df_clean.columns)
new_features = [col for col in df_featured.columns if col not in original_features]

print(f"Total new features created: {len(new_features)}")
print("\nNew Features:")
print("=" * 80)
for i, feature in enumerate(new_features, 1):
    print(f"{i:2d}. {feature}")

In [ ]:
# Show summary statistics for new numerical features
new_numerical_features = [
    'avg_payment_status', 'max_payment_delay', 'num_delayed_payments',
    'avg_bill_amt', 'credit_utilization', 'payment_to_bill_ratio',
    'avg_payment_amt', 'num_zero_payments'
]

print("\nSummary Statistics for Key Engineered Features:")
print("=" * 80)
df_featured[new_numerical_features].describe()

### 7.8 Analyze Feature Importance (Correlation with Target)

In [ ]:
# Calculate correlation of new features with target
feature_correlations = df_featured[new_numerical_features + ['default']].corr()['default'].drop('default')
feature_correlations = feature_correlations.sort_values(ascending=False)

print("Correlation of Engineered Features with Default:")
print("=" * 80)
print(feature_correlations)

# Visualize
plt.figure(figsize=(10, 6))
feature_correlations.plot(kind='barh', color=plt.cm.RdYlGn_r(np.abs(feature_correlations)))
plt.title('Feature Correlation with Default', fontsize=14, fontweight='bold')
plt.xlabel('Correlation Coefficient')
plt.ylabel('Feature')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

## 8. Fairness Analysis Preparation

### 8.1 Protected Attributes Analysis

For fair ML, we need to understand how our model might treat different demographic groups. Common protected attributes in financial contexts include gender, age, and marital status.

In [ ]:
# Analyze default rates across protected groups
print("Default Rates by Protected Attributes:")
print("=" * 80)

# By Gender
print("\n1. By Gender:")
gender_default = df_featured.groupby('SEX')['default'].agg(['mean', 'count'])
gender_default.index = ['Male', 'Female']
gender_default.columns = ['Default Rate', 'Count']
print(gender_default)

# By Education
print("\n2. By Education:")
edu_default = df_featured.groupby('EDUCATION')['default'].agg(['mean', 'count'])
edu_default.index = ['Graduate School', 'University', 'High School', 'Others']
edu_default.columns = ['Default Rate', 'Count']
print(edu_default)

# By Marriage
print("\n3. By Marital Status:")
mar_default = df_featured.groupby('MARRIAGE')['default'].agg(['mean', 'count'])
mar_default.index = ['Married', 'Single', 'Others']
mar_default.columns = ['Default Rate', 'Count']
print(mar_default)

# By Age Group
print("\n4. By Age Group:")
age_default = df_featured.groupby('age_group')['default'].agg(['mean', 'count'])
age_default.columns = ['Default Rate', 'Count']
print(age_default)

In [ ]:
# Visualize fairness metrics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Gender
gender_default.plot(kind='bar', y='Default Rate', ax=axes[0, 0], 
                    color=['#3498db', '#e91e63'], legend=False)
axes[0, 0].set_title('Default Rate by Gender', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Default Rate')
axes[0, 0].set_xticklabels(['Male', 'Female'], rotation=0)

# Education
edu_default.plot(kind='bar', y='Default Rate', ax=axes[0, 1], legend=False)
axes[0, 1].set_title('Default Rate by Education', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Default Rate')
axes[0, 1].tick_params(axis='x', rotation=45)

# Marriage
mar_default.plot(kind='bar', y='Default Rate', ax=axes[1, 0], 
                 color=['#9b59b6', '#f39c12', '#34495e'], legend=False)
axes[1, 0].set_title('Default Rate by Marital Status', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Default Rate')
axes[1, 0].tick_params(axis='x', rotation=0)

# Age Group
age_default.plot(kind='bar', y='Default Rate', ax=axes[1, 1], legend=False)
axes[1, 1].set_title('Default Rate by Age Group', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Default Rate')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 8.2 Statistical Significance Testing

In [ ]:
# Chi-square tests for independence
print("Chi-Square Tests for Independence:")
print("=" * 80)

protected_attrs = [
    ('SEX', 'Gender'),
    ('EDUCATION', 'Education'),
    ('MARRIAGE', 'Marital Status')
]

for attr, name in protected_attrs:
    contingency_table = pd.crosstab(df_featured[attr], df_featured['default'])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    print(f"\n{name}:")
    print(f"  Chi-square statistic: {chi2:.4f}")
    print(f"  P-value: {p_value:.6f}")
    print(f"  Degrees of freedom: {dof}")
    
    if p_value < 0.05:
        print(f"  ⚠️  Significant association found (p < 0.05)")
    else:
        print(f"  ✓ No significant association (p >= 0.05)")

### 8.3 Disparate Impact Analysis

In [ ]:
# Calculate disparate impact ratio (80% rule)
# Ratio should be >= 0.8 to avoid disparate impact

print("Disparate Impact Analysis (80% Rule):")
print("=" * 80)
print("A ratio < 0.8 suggests potential disparate impact\n")

# Gender
male_default_rate = df_featured[df_featured['SEX'] == 1]['default'].mean()
female_default_rate = df_featured[df_featured['SEX'] == 2]['default'].mean()
gender_di_ratio = min(male_default_rate, female_default_rate) / max(male_default_rate, female_default_rate)

print(f"Gender:")
print(f"  Male default rate: {male_default_rate:.4f}")
print(f"  Female default rate: {female_default_rate:.4f}")
print(f"  Disparate Impact Ratio: {gender_di_ratio:.4f}")
if gender_di_ratio < 0.8:
    print(f"  ⚠️  Potential disparate impact detected!")
else:
    print(f"  ✓ No disparate impact")

# Age groups
print(f"\nAge Groups:")
age_group_rates = df_featured.groupby('age_group')['default'].mean()
age_di_ratio = age_group_rates.min() / age_group_rates.max()
print(age_group_rates)
print(f"  Disparate Impact Ratio: {age_di_ratio:.4f}")
if age_di_ratio < 0.8:
    print(f"  ⚠️  Potential disparate impact detected!")
else:
    print(f"  ✓ No disparate impact")

## 9. Prepare Final Dataset for Modeling

In [ ]:
# Select features for modeling
print("Preparing final dataset for modeling...")

# Original features to keep
original_to_keep = [
    'LIMIT_BAL', 'AGE',
    'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
    'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
    'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'
]

# Engineered features to keep
engineered_to_keep = [
    # Payment behavior
    'avg_payment_status', 'max_payment_delay', 'num_delayed_payments',
    'num_ontime_payments', 'payment_status_trend', 'recent_payment_status',
    # Credit utilization
    'avg_bill_amt', 'max_bill_amt', 'bill_amt_volatility',
    'credit_utilization', 'recent_credit_utilization', 'bill_amt_trend',
    # Payment capacity
    'avg_payment_amt', 'payment_to_bill_ratio', 'recent_payment_to_bill',
    'payment_amt_volatility', 'num_zero_payments', 'payment_consistency',
    # Demographic (encoded)
    'SEX_male', 'SEX_female',
    'EDU_graduate', 'EDU_university', 'EDU_high_school', 'EDU_others',
    'MAR_married', 'MAR_single', 'MAR_others',
    # Age groups
    'age_<25', 'age_25-35', 'age_35-45', 'age_45-55', 'age_55+',
    'credit_per_age',
    # Interactions
    'high_util_delayed', 'low_payment_high_bill'
]

# Protected attributes (for fairness monitoring - not for training)
protected_attributes = ['SEX', 'EDUCATION', 'MARRIAGE', 'age_group']

# Target
target = ['default']

# Combine all
all_features = original_to_keep + engineered_to_keep
all_columns = ['ID'] + protected_attributes + all_features + target

# Create final dataset
df_final = df_featured[all_columns].copy()

print(f"✓ Final dataset created")
print(f"  Shape: {df_final.shape}")
print(f"  Features: {len(all_features)}")
print(f"  Protected attributes: {len(protected_attributes)}")

In [ ]:
# Display final dataset info
print("\nFinal Dataset Information:")
print("=" * 80)
df_final.info()

In [ ]:
# Check for any remaining missing values or infinities
print("\nData Quality Check:")
print("=" * 80)

missing = df_final.isnull().sum().sum()
print(f"Missing values: {missing}")

# Check for infinite values in numerical columns
numerical_cols = df_final.select_dtypes(include=[np.number]).columns
infinite_counts = np.isinf(df_final[numerical_cols]).sum().sum()
print(f"Infinite values: {infinite_counts}")

if missing == 0 and infinite_counts == 0:
    print("\n✓ Data quality check passed!")
else:
    print("\n⚠️  Data quality issues detected. Please review.")

## 10. Save Prepared Dataset

In [ ]:
# Save the prepared dataset
output_path = 'credit_card_default_prepared.csv'
df_final.to_csv(output_path, index=False)

print(f"✓ Prepared dataset saved to: {output_path}")
print(f"  Rows: {df_final.shape[0]:,}")
print(f"  Columns: {df_final.shape[1]}")

In [ ]:
# Create a data dictionary for reference
data_dictionary = pd.DataFrame({
    'Feature': all_features,
    'Category': (
        ['Credit'] * 1 + ['Demographic'] * 1 +
        ['Payment History'] * 6 +
        ['Bill Amount'] * 6 +
        ['Payment Amount'] * 6 +
        ['Engineered - Payment Behavior'] * 6 +
        ['Engineered - Credit Utilization'] * 6 +
        ['Engineered - Payment Capacity'] * 6 +
        ['Engineered - Demographics'] * 11 +
        ['Engineered - Age'] * 6 +
        ['Engineered - Interaction'] * 2
    )
})

# Save data dictionary
dict_path = 'feature_dictionary.csv'
data_dictionary.to_csv(dict_path, index=False)
print(f"\n✓ Feature dictionary saved to: {dict_path}")

## 11. Summary and Next Steps

In [ ]:
print("="*80)
print("DATA PREPARATION & FEATURE ENGINEERING COMPLETE")
print("="*80)

print("\n📊 Dataset Overview:")
print(f"  • Total samples: {df_final.shape[0]:,}")
print(f"  • Total features: {len(all_features)}")
print(f"  • Original features: {len(original_to_keep)}")
print(f"  • Engineered features: {len(engineered_to_keep)}")
print(f"  • Protected attributes tracked: {len(protected_attributes)}")

print("\n🎯 Target Variable:")
print(f"  • Class distribution:")
print(f"    - No default: {(df_final['default']==0).sum():,} ({(df_final['default']==0).mean()*100:.1f}%)")
print(f"    - Default: {(df_final['default']==1).sum():,} ({(df_final['default']==1).mean()*100:.1f}%)")

print("\n🛡️ Fairness Considerations:")
print(f"  • Protected attributes identified and analyzed")
print(f"  • Disparate impact ratios calculated")
print(f"  • Statistical significance tests performed")

print("\n📁 Output Files:")
print(f"  • Prepared dataset: {output_path}")
print(f"  • Feature dictionary: {dict_path}")

print("\n🚀 Next Steps:")
print("  1. Build baseline classical ML models (Logistic Regression, Random Forest, XGBoost)")
print("  2. Implement cost-sensitive learning (accounting for false negative/positive costs)")
print("  3. Apply fairness-aware algorithms and mitigation strategies")
print("  4. Develop deep learning models with interpretability")
print("  5. Use SHAP/LIME for model explainability")
print("  6. Evaluate models on both performance and fairness metrics")
print("  7. Compare cost-aware vs standard approaches")

print("\n" + "="*80)

---

## Key Insights from Data Preparation

### 1. **Class Imbalance**
   - The dataset shows class imbalance favoring non-defaulters
   - Will require techniques like SMOTE, class weights, or cost-sensitive learning

### 2. **Feature Engineering Impact**
   - Created 30+ engineered features capturing:
     - Payment behavior patterns
     - Credit utilization trends
     - Payment capacity indicators
   - These features show stronger correlation with default than many raw features

### 3. **Fairness Considerations**
   - Identified potential disparate impact across demographic groups
   - Protected attributes (gender, age, education) will need careful monitoring
   - Must ensure models don't discriminate based on these attributes

### 4. **Data Quality**
   - No missing values in dataset
   - Clean categorical variables
   - Outliers present but may represent genuine cases

### 5. **Cost-Aware Considerations**
   - False negatives (missing defaults) are more costly than false positives
   - Will need to adjust decision thresholds and use appropriate cost matrices
   - Business impact should drive model evaluation metrics

---

**Ready for modeling!** The prepared dataset is now suitable for building cost-aware, fair, and explainable ML models.